In [1]:
#importing the required libraries
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#loading the drivers and URL

driver=webdriver.Chrome('chromedriver.exe')
driver.get('http://amazon.in/')


# Creating the inpiut for the product name to be entered
product=input("Enter the product name: ")

#updating the input details in seracg=h bar
driver.find_element_by_id("twotabsearchtextbox").send_keys(product)
driver.find_element_by_id("nav-search-submit-button").click()

Enter the product name: Laptop


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
#to update the url of all products from 3 pages
urls=[]
for i in range(3):
    for j in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(j.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break

In [4]:
#creating the empty list
brand=[]
name=[]
rating=[]
num_of_rating=[]
price=[]
return_exc=[]
exp_del=[]
avail=[]
pro_url=[]

#updating all the prodcut urls
for i in urls:
    driver.get(i)
    pro_url.append(i)
    time.sleep(2)
    
#Scrapping the brand name
    try:
        brand.append(driver.find_element_by_id("productTitle").text.split(' ')[0])
        
    except NoSuchElementException:
        brand.append("-")
            
#Scrapping the prodcut name
    try:
        name.append(driver.find_element_by_id("productTitle").text.split(' ')[1:]) 
    except NoSuchElementException:
        name.append("-")
            
#Scrapping the product rating
    try:
        rating.append(driver.find_element_by_xpath('//*[@id="a-popover-content-8"]/div/div/div/div[1]/span').text)
    except NoSuchElementException:
        rating.append("-")

#Scrapping the number of rating
    try:
        num_of_rating.append(driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text)
    except NoSuchElementException:
        num_of_rating.append("-")
    
#Scrapping the price
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
            
#scrapping for return/exchange
    try:
        return_exc.append(driver.find_element_by_xpath('//*[@id="RETURNS_POLICY"]/span/div[2]/a').text)
    except NoSuchElementException:
        return_exc.append("-")
            
#scrapping for expected delivery
    try:
        exp_del.append(driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text)
    except NoSuchElementException:
        exp_del.append("-")

#scrapping for availability
    try:
        avail.append(driver.find_element_by_id("availability").text)
    except NoSuchElementException:
        avail.append("-")

In [5]:
#creating the DataFrame
Product={'Brand':brand,'Name':name,'Rating':rating,'Number of rating':num_of_rating,'Price':price,
        'Return/Exchange':return_exc,'Expected Delivery':exp_del,'Availability':avail,'URL':pro_url}
Product=pd.DataFrame(data=Product)
Product

,Brand,Name,Rating,Number of rating,Price,Return/Exchange,Expected Delivery,Availability,URL
0,LG,"[Gram, 16, Ultra-Light, 11th, Gen, Core, i5,8,...",-,85 ratings,"₹99,492.00",7 Days Replacement,"Friday, Oct 1",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,LG,"[Gram, 10th, Gen, Intel, Core, i7-1065G7, 17-i...",-,52 ratings,"₹1,17,027.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Acer,"[Aspire, 3, AMD, Athlon, Silver, 3050U, Dual-c...",-,5 ratings,"₹32,500.00",7 Days Replacement,"Wednesday, Sep 29",Only 2 left in stock.,https://www.amazon.in/Acer-Aspire-Athlon-Dual-...
3,Dell,"[Inspiron, 3502, 15.6"", (39.62, cms), HD, Disp...",-,225 ratings,"₹40,126.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,https://www.amazon.in/Dell-Inspiron-Integrated...
4,HP,"[Chromebook, 14-inch, (35.56, cms), Thin, &, L...",-,775 ratings,"₹27,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,https://www.amazon.in/HP-Chromebook-14a-na0003...
...,...,...,...,...,...,...,...,...,...
65,(Renewed),"[Fujitsu, LifeBook, 14, inch, (35.56, cm), HD,...",-,-,"₹22,999.00",7 Days Replacement,"Thursday, Sep 30",In stock.,https://www.amazon.in/Fujitsu-LifeBook-Busines...
66,HP,"[Pavilion, (2021), Thin, &, Light, 11th, Gen, ...",-,164 ratings,"₹67,990.00",7 Days Replacement,"Thursday, Sep 30",Only 1 left in stock.,https://www.amazon.in/HP-Pavilion-Laptop-Graph...
67,AVITA,"[Essential, Refresh, NE14A2INC44A-CR, Intel, C...",-,89 ratings,"₹25,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,https://www.amazon.in/AVITA-Essential-NE14A2IN...
68,Del,"[Inspiron, 3501, 15.6, inch, (39.6, cm), FHD, ...",-,62 ratings,"₹67,888.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [6]:
# We will create empty list in which we will update the required data
fruits=[]
cars=[]
machine=[]

#creating the list for the name of the items we want the images
Name=['fruits','cars','machine']

#loading the drivers and url
driver=webdriver.Chrome("chromedriver.exe")
driver.maximize_window()
wait = WebDriverWait(driver,15)
driver.get("https://www.google.co.in/imghp?hl=en&ogbl")

for i in Name:
    if i=='fruits':
        driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i) #entering the item for search
        driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()

#creating the instance to scroll down the page for images to load.
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

#scrapping for fruits            
    fruit=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in fruit:
        fruits.append(k.get_attribute('href'))
    html.send_keys(Keys.HOME)
    driver.find_element_by_xpath("//input[@class='og3lId']").clear() #clearing the data in searc bar

#scrapping for cars
    if i=='cars':
        driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
        driver.find_element_by_xpath("//button[@class='rCGXm']").click()
        
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
    car=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in car:
        cars.append(k.get_attribute('href'))
    html.send_keys(Keys.HOME)
    driver.find_element_by_xpath("//input[@class='og3lId']").clear()

#scrapping for machine

    if i=='machine':
        driver.find_element_by_xpath("//input[@class='og3lId']").send_keys(i)
        driver.find_element_by_xpath("//button[@class='rCGXm']").click()
        
        SCROLL_PAUSE_TIME = 2
        CYCLES = 10

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN)  
        time.sleep(SCROLL_PAUSE_TIME * 3)

        for j in range(CYCLES):
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)
    mac=driver.find_elements_by_xpath("//a[@class='VFACy kGQAp sMi44c lNHeqe WGvvNb']")
    for k in mac:
        machine.append(k.get_attribute('href'))

In [7]:
#creating the dataframe
Google={'Fruits':fruits,'Cars':cars,'Machine':machine}
Google=pd.DataFrame(data=Google)
Google[:100]

,Fruits,Cars,Machine
0,https://www.amazon.in/Fruits-All-One-Dreamland...,https://www.amazon.in/Fruits-All-One-Dreamland...,https://www.amazon.in/Fruits-All-One-Dreamland...
1,https://www.healthyeating.org/nutrition-topics...,https://www.healthyeating.org/nutrition-topics...,https://www.healthyeating.org/nutrition-topics...
2,https://www.youtube.com/watch?v=PY2OxP_jMZ0,https://www.youtube.com/watch?v=PY2OxP_jMZ0,https://www.youtube.com/watch?v=PY2OxP_jMZ0
3,https://in.pinterest.com/pin/649081365039047796/,https://in.pinterest.com/pin/649081365039047796/,https://in.pinterest.com/pin/649081365039047796/
4,https://www.goodhousekeeping.com/health/diet-n...,https://www.goodhousekeeping.com/health/diet-n...,https://www.goodhousekeeping.com/health/diet-n...
...,...,...,...
95,https://www.thespruceeats.com/thai-preparation...,https://www.thespruceeats.com/thai-preparation...,https://www.thespruceeats.com/thai-preparation...
96,https://www.mysouthernhealth.com/9-refreshing-...,https://www.mysouthernhealth.com/9-refreshing-...,https://www.mysouthernhealth.com/9-refreshing-...
97,https://www.nutritionadvance.com/healthy-foods...,https://www.nutritionadvance.com/healthy-foods...,https://www.nutritionadvance.com/healthy-foods...
98,https://www.amazon.com/TREND-enterprises-Fruit...,https://www.amazon.com/TREND-enterprises-Fruit...,https://www.amazon.com/TREND-enterprises-Fruit...


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [8]:
#loading the driver and the url
driver=webdriver.Chrome(r"C:\Users\dheerajkumar_pittala\DataPy\Browserdrivers\chromedriver.exe")
time.sleep(2)
url=('https://www.flipkart.com/')
driver.get(url)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
time.sleep(2)

In [9]:
#creating the user input to search for mobiles
user_inp= input('Enter the product you want to search: ')
driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input").send_keys(user_inp)
time.sleep(2)
driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
time.sleep(2)

Enter the product you want to search: Smartphone


In [10]:
#creating the empty list
Brand=[] 
Smartphone_name=[]
Colour=[]
RAM=[] 
Storage=[]
Primary_Camera=[]
Secondary_Camera=[]
Display_Size=[]
Display_Resolution=[]
Processor=[]
Processor_Cores=[]
Battery_Capacity=[]
Price=[]
Product_url=[]

In [11]:
#clicking on the product link
product_links=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

#scrapping the product link
for i in product_links:
    Product_url.append(i.get_attribute('href'))
for i in Product_url:
    driver.get(i)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
#scrapping the Brand name
    try:
        brand=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        Brand.append(brand[0])
    except:
        Brand.append('-')
#scrapping other details
    a= driver.find_elements_by_xpath("//li[@class='_21lJbe']")
    b=driver.find_elements_by_xpath("//td[@class='_1hKmbr col col-3-12']")
    for i in range(0,len(b)):
        if b[i].text=='Model Name':
            Smartphone_name.append(a[i].text)        
        if b[i].text=='Color':
            Colour.append(a[i].text)
        if b[i].text=='RAM':
            RAM.append(a[i].text)
        if b[i].text=='Internal Storage':
            Storage.append(a[i].text)
        if b[i].text=='Primary Camera':
            Primary_Camera.append(a[i].text)
        if b[i].text=='Secondary Camera':
            Secondary_Camera.append(a[i].text)                                  
        if b[i].text=='Display Size':
            Display_Size.append(a[i].text)
        if b[i].text=='Resolution':
            Display_Resolution.append(a[i].text)
        if b[i].text=='Processor Type':
            Processor.append(a[i].text)            
        if b[i].text=='Processor Core':
            Processor_Cores.append(a[i].text)
        if b[i].text=='Battery Capacity':
            Battery_Capacity.append(a[i].text)
        if b[i].text=='Product url':
            Product_url.append(a[i].text)
#scrapping the prices
    try:
        price=driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text)
    except:
        Price.append('-')

In [12]:
#creating the dataframe
Details={'Brand Name':Brand,'Smartphone Name':Smartphone_name,'Color':Colour,'RAM':RAM,'Storage(ROM)':Storage,
        'Primary Camera':Primary_Camera,'Secondary Camera':Secondary_Camera,'Display Size':Display_Size,
         'Display Resolution':Display_Resolution,'Processor':Processor,'Processor Cores':Processor_Cores,
        'Battery Capacity':Battery_Capacity,'Price':Price,'Product_url':Product_url}

#Converting the details into dataframe

Details=pd.DataFrame(data=Details)
Details

,Brand Name,Smartphone Name,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product_url
0,POCO,C3,Matte Black,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹8,999",https://www.flipkart.com/poco-c3-matte-black-6...
1,POCO,C3,Arctic Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,Mediatek Helio G35,Octa Core,5000 mAh,"₹8,999",https://www.flipkart.com/poco-c3-arctic-blue-6...
2,realme,RMX3063 / RMX3061,C20,32 GB,"2G GSM: 850/900/1800/1900 MHz, 3G WCDMA: 850/9...",Yes,Yes,"Head: 0.982W/kg, Body: 0.497W/kg",16.51 cm (6.5 inch),Android 10,MediaTek Helio G35,"MP4, 3GP, 3G2, 3GPP, 3GPP2, M4V, MKV","₹7,499",https://www.flipkart.com/realme-c20-cool-blue-...
3,realme,RMX3063 / RMX3061,C20,32 GB,"2G GSM: 850/900/1800/1900 MHz, 3G WCDMA: 850/9...",Yes,Yes,"Head: 0.982W/kg, Body: 0.497W/kg",16.51 cm (6.5 inch),Android 10,MediaTek Helio G35,"MP4, 3GP, 3G2, 3GPP, 3GPP2, M4V, MKV","₹7,499",https://www.flipkart.com/realme-c20-cool-grey-...
4,REDMI,9i,Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,"₹8,799",https://www.flipkart.com/redmi-9i-nature-green...
5,realme,C21Y,Cross Black,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610,Octa Core,5000 mAh,"₹9,999",https://www.flipkart.com/realme-c21y-cross-bla...
6,realme,C21Y,Cross Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610,Octa Core,5000 mAh,"₹9,999",https://www.flipkart.com/realme-c21y-cross-blu...
7,realme,C11 2021,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,5000 mAh,"₹7,299",https://www.flipkart.com/realme-c11-2021-cool-...
8,realme,C11 2021,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,5000 mAh,"₹7,299",https://www.flipkart.com/realme-c11-2021-cool-...
9,POCO,M2 Reloaded,Mostly Blue,4 GB,64 GB,13MP + 8MP + 5MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5000 mAh,"₹9,999",https://www.flipkart.com/poco-m2-reloaded-most...


In [13]:
#Saving the Details
Details.to_csv("Smartphones_flipkart.csv")

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [14]:
driver=webdriver.Chrome(r"C:\Users\dheerajkumar_pittala\DataPy\Browserdrivers\chromedriver.exe")
time.sleep(2)
driver.get('https://www.google.com/maps/')

City = input('Enter City Name : ')
driver.find_element_by_xpath("//input[@class='tactile-searchbox-input']").send_keys(City)
driver.find_element_by_xpath("//button[@class='searchbox-searchbutton']").click()

latitude=driver.current_url.split('/@')[1].split(',')[0]
longtitude=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',latitude,'Longitude:',longtitude)

Enter City Name : Bangalore
Latitude: 17.5079424 Longitude: 78.3908864


# 6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [15]:
#loading the driver and the url
driver=webdriver.Chrome('chromedriver.exe')
driver.get('https://trak.in/')
time.sleep(2)

driver.get(driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href'))

In [16]:
#creating the empty list 
date=[]
name=[]
industry=[]
sub=[]
city=[]
investor=[]
inv_type=[]
amount=[]

In [17]:
#scrapping for date
for i in driver.find_elements_by_xpath('//td[@class="column-2"]'):
    date.append(i.text)

#scrapping for name
for i in driver.find_elements_by_xpath('//td[@class="column-3"]'):
    name.append(i.text)

#scrapping for industry    
for i in driver.find_elements_by_xpath('//td[@class="column-4"]'):
    industry.append(i.text)

#scrapping for sub industry
for i in driver.find_elements_by_xpath('//td[@class="column-5"]'):
    sub.append(i.text)

#scrapping for city    
for i in driver.find_elements_by_xpath('//td[@class="column-6"]'):
    city.append(i.text)

#scrapping for investor
for i in driver.find_elements_by_xpath('//td[@class="column-7"]'):
    investor.append(i.text)

#scrapping for investor type    
for i in driver.find_elements_by_xpath('//td[@class="column-8"]'):
    inv_type.append(i.text)

#scrapping for amount
for i in driver.find_elements_by_xpath('//td[@class="column-9"]'):
    amount.append(i.text)

In [18]:
#checking for length of each columns
len(date),len(name),len(industry),len(sub),len(city),len(investor),len(inv_type),len(amount)

(392, 392, 392, 392, 392, 392, 392, 392)

In [19]:
#creating the dataframe for Jul-Sep
Funding={'Date':date,'Name':name,'Industry/Vertica':industry,'Sub-vertical':sub,'City':city,'Investor':investor,
        'Investment Type':inv_type,'Amount':amount}
Funding=pd.DataFrame(data=Funding)
Funding[55:85]

,Date,Name,Industry/Vertica,Sub-vertical,City,Investor,Investment Type,Amount
55,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
56,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
57,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
58,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
59,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
60,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
61,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
62,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
63,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
64,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [20]:
driver= webdriver.Chrome(r"C:\Users\dheerajkumar_pittala\DataPy\Browserdrivers\chromedriver.exe")
driver.get("https://www.digit.in/")

In [21]:
menu=driver.find_elements_by_xpath("//div[@class='menu']/ul/li[3]/a")
for i in menu:
    url=i.get_attribute("href")
    driver.get(url)
    toplaptop=driver.find_elements_by_xpath("//div[@class='Listbrand']/ul/li[10]/a")
    for j in toplaptop:
        url1=j.get_attribute("href")
        print(url1)
        driver.get(url1)

https://www.digit.in/top-products/best-gaming-laptops-40.html


In [22]:
#getting gaming laptop names
laptopnameList=[]
p_n=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in p_n:
    laptopnameList.append(i.text)
laptopnameList

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [23]:
#getting specs
OS_List=[]
display_List=[]
processor_List=[]
memory_list=[]
weight_list=[]
dimension_list=[]
graphicprocessor_list=[]
#Getting OS
os=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]")
for i in os:
    OS_List.append(i.text)
OS_List

#getting display specs
display=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
for i in display:
    display_List.append(i.text)
display_List


#getting processor specs
processor=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
for i in processor :
    processor_List.append(i.text)
processor_List


#getting memory specs
memory=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
for i in memory :
    memory_list.append(i.text)
memory_list

#getting weight specs
weight=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
for i in weight :
    weight_list.append(i.text)
weight_list

#getting dimension specs
dimensions=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
for i in dimensions :
    dimension_list.append(i.text)
dimension_list

#getting graphic processor specs
graphic=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]")
for i in graphic :
    graphicprocessor_list.append(i.text)
graphicprocessor_list

['Intel® UHD Graphics 630',
 'NA',
 'NVIDIA® GeForce RTX™ 3070',
 'NVIDIA GeForce RTX 2060',
 'NVIDIA® GeForce® GTX 1650 4GB',
 'NVIDIA GeForce RTX 2070 Max-Q',
 'NVIDIA® GeForce® GTX 1650']

In [24]:
#finding all the price tags
price = driver.find_elements_by_class_name("smprice")
price[0:3]
#scraping price from price tags
PRICE=[]
for i in price:
    prices=i.text.replace("\n"," ")
    PRICE.append(prices)
PRICE

['N/A', '₹ 341,990', 'N/A', '₹ 164,990', '₹ 76,988', '₹ 185,000', '₹ 64,370']

In [25]:
#finding all the seller tags
seller = driver.find_elements_by_class_name("smmerchant")
seller[0:3]

#scraping Seller from seller tags
Seller=[]
for i in seller:
    sellers=i.text.replace("\n"," ")
    Seller.append(sellers)
Seller

['N/A', 'N/A', 'N/A', 'Amazon', 'Amazon', 'Amazon', 'Amazon']

In [26]:
import pandas as pd

#Converting the data into dataframe

TopLaptop=pd.DataFrame({})
TopLaptop['Laptop Model Name']=laptopnameList
TopLaptop['OS']=OS_List
TopLaptop['Display']=display_List
TopLaptop['Processor']=processor_List
TopLaptop['Memory']=memory_list
TopLaptop['Weight']=weight_list
TopLaptop['Dimension']=dimension_list
TopLaptop['Graphics Processor']=graphicprocessor_list
TopLaptop['Seller']=Seller
TopLaptop['Price']=PRICE
TopLaptop

,Laptop Model Name,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Seller,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A,N/A
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,N/A,"₹ 341,990"
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A,N/A
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,Amazon,"₹ 164,990"
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,Amazon,"₹ 76,988"
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,Amazon,"₹ 185,000"
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,Amazon,"₹ 64,370"


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [27]:
def driver_launch(url):
    driver_path = '/Users/dheerajkumar_pittala/DataPy/Browserdrivers/chromedriver'
    driver = webdriver.Chrome(executable_path=driver_path)
    
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get(url)
    
    return driver

In [28]:
def get_forbes(url):
    #file_path = "files/forbes-billioniares.csv"
    ranks = []
    names = []
    net_worths = []
    ages = []
    citizenships = []
    sources = []
    industries = []

    d = driver_launch(url)
    time.sleep(5)
    
    name_elems = d.find_elements_by_xpath("//div[@class='personName']")
    rank_elems = d.find_elements_by_xpath("//div[@class='rank']")
    net_elems = d.find_elements_by_xpath("//div[@class='netWorth']")
    age_elems = d.find_elements_by_xpath("//div[@class='age']")
    citi_elems = d.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    source_elems = d.find_elements_by_xpath("//div[@class='source']")
    cate_elems = d.find_elements_by_xpath("//div[@class='category']")
    
    for name_elems, rank_elems, net_elems, age_elems, citi_elems, source_elems, cate_elems in zip(name_elems, rank_elems, net_elems, age_elems, citi_elems, source_elems, cate_elems):
        names.append(name_elems.text)
        ranks.append(rank_elems.text.replace(".", ""))
        net_worths.append(net_elems.text)
        ages.append(age_elems.text)
        citizenships.append(citi_elems.text)
        sources.append(source_elems.text)
        industries.append(cate_elems.text)
                                          
    df = pd.DataFrame({'Name':names, 'Net Worth':net_worths, 'Rank':ranks, 'Age':ages, 
                              'Country/Citizenship':citizenships, 'Source':sources, 'Industries': industries
                             })
    #df.to_csv(file_path, sep=',')
    print(f"Saved data to csv file: files/forbes-billioniares.csv")

    return d, df

In [29]:
d, data = get_forbes("https://www.forbes.com/billionaires/")
d.quit()

Saved data to csv file: files/forbes-billioniares.csv


In [30]:
data

,Name,Net Worth,Rank,Age,Country/Citizenship,Source,Industries
0,Jeff Bezos,$177 B,1,57,United States,Amazon,Technology
1,Elon Musk,$151 B,2,49,United States,"Tesla, SpaceX",Automotive
2,Bernard Arnault & family,$150 B,3,72,France,LVMH,Fashion & Retail
3,Bill Gates,$124 B,4,65,United States,Microsoft,Technology
4,Mark Zuckerberg,$97 B,5,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,Harry Triguboff,$11.2 B,195,88,Australia,real estate,Real Estate
196,Leonid Fedun & family,$11.1 B,197,65,Russia,oil,Energy
197,Eyal Ofer,$11.1 B,197,70,Israel,"real estate, shipping",Diversified
198,Evan Spiegel,$11.1 B,197,30,United States,Snapchat,Technology


In [31]:
data.head(10)

,Name,Net Worth,Rank,Age,Country/Citizenship,Source,Industries
0,Jeff Bezos,$177 B,1,57,United States,Amazon,Technology
1,Elon Musk,$151 B,2,49,United States,"Tesla, SpaceX",Automotive
2,Bernard Arnault & family,$150 B,3,72,France,LVMH,Fashion & Retail
3,Bill Gates,$124 B,4,65,United States,Microsoft,Technology
4,Mark Zuckerberg,$97 B,5,36,United States,Facebook,Technology
5,Warren Buffett,$96 B,6,90,United States,Berkshire Hathaway,Finance & Investments
6,Larry Ellison,$93 B,7,76,United States,software,Technology
7,Larry Page,$91.5 B,8,48,United States,Google,Technology
8,Sergey Brin,$89 B,9,47,United States,Google,Technology
9,Mukesh Ambani,$84.5 B,10,63,India,diversified,Diversified


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.


In [32]:
#importing the required libraries
import pandas as pd
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [33]:
#creating the empty list
Comments=[]
Upvote=[]
Updated=[]

#loading the drivers and url
driver=webdriver.Chrome(r"C:\Users\dheerajkumar_pittala\DataPy\Browserdrivers\chromedriver.exe")
wait = WebDriverWait(driver,15)
driver.get("https://www.youtube.com/watch?v=ukzFI9rgwfU")

#creating the program to scroll down the comments and let it load
SCROLL_PAUSE_TIME = 2
CYCLES = 100

html = driver.find_element_by_tag_name('html')
html.send_keys(Keys.PAGE_DOWN)  
html.send_keys(Keys.PAGE_DOWN)  
time.sleep(SCROLL_PAUSE_TIME * 3)

for i in range(CYCLES):
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)

#scrapping for comments
comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
for i in comment:
    Comments.append(i.text)

#scrapping for upvote
upvote = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
for i in upvote:
    Upvote.append(i.text)

#scrapping for updated on
updated=driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for i in updated:
    Updated.append(i.text)

#creating the dataframe
Youtube={'Comments':Comments,'Comment Upvote':Upvote, 'Updated on':Updated}
Youtube=pd.DataFrame(data=Youtube)
Youtube

,Comments,Comment Upvote,Updated on
0,🔥Explore Our FREE Courses With Completion Cert...,8,3 weeks ago
1,**Summary**:\n- machine learning is the genera...,2.8K,2 years ago
2,Scenario 1: Facebook recognizes your friend in...,1K,2 years ago
3,I'm impressed by the way you taught. Teacher s...,229,1 year ago
4,Literally learnt more from you than 4 years in...,129,4 months ago
...,...,...,...
602,too fasttt video was,,9 months ago
603,you should have use a better voice over withou...,1,2 years ago
604,Alexa,,1 year ago
605,1. Unsupervised\n2. Supervised \n3. Unsupervised,,2 years ago


In [34]:
Youtube.head(500)

,Comments,Comment Upvote,Updated on
0,🔥Explore Our FREE Courses With Completion Cert...,8,3 weeks ago
1,**Summary**:\n- machine learning is the genera...,2.8K,2 years ago
2,Scenario 1: Facebook recognizes your friend in...,1K,2 years ago
3,I'm impressed by the way you taught. Teacher s...,229,1 year ago
4,Literally learnt more from you than 4 years in...,129,4 months ago
...,...,...,...
495,Very informative video mam,,1 year ago
496,What I learned: don’t use google or Facebook! ...,1,2 years ago
497,👍👍👍,,8 months ago
498,Pls share learning guide.,1,1 year ago


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [35]:
def driver_launch(url):
    driver_path = '/Users/dheerajkumar_pittala/DataPy/Browserdrivers/chromedriver'
    driver = webdriver.Chrome(executable_path=driver_path)
    
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get(url)
    
    return driver

In [36]:
def get_hostels_test(url):
    file_path = "files/hostels-london.csv"
    num_pages = 4
    names = []
    distance = []
    ratings = []
    t_rev = []
    o_rev = []
    privates = []
    dorms = []
    faci = []
    desc = []
    
    for i in range (1, num_pages):
        d = driver_launch(url+f"s?q=London,%20England&country=England&city=London&type=city&id=3&from=2021-06-23&to=2021-06-24&page={i}")
        time.sleep(10)

        name_elems = d.find_elements_by_xpath('//h2[@class="title title-6"]')
        dist_elems = d.find_elements_by_xpath('//span[@class="description"]')
        rate_elems = d.find_elements_by_xpath('//div[@class="score orange big"]')
        tr_elems = d.find_elements_by_xpath('//div[@class="reviews"]')
        or_elems = d.find_elements_by_xpath('//div[@class="rating rating-summary-container big"]/div/div/span')
        pr_elems = d.find_elements_by_xpath('//a[@class="prices"]/div[1]/div')
        dr_elems = d.find_elements_by_xpath('//a[@class="prices"]/div[2]/div')
        fac_elems = d.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
        de1_elems = d.find_elements_by_xpath('//div[@class="rating-factors prop-card-tablet rating-factors small"]/div[1]')
        de2_elems = d.find_elements_by_xpath('//div[@class="rating-factors prop-card-tablet rating-factors small"]/div[2]')
        de3_elems = d.find_elements_by_xpath('//div[@class="rating-factors prop-card-tablet rating-factors small"]/div[3]')
        time.sleep(4)

    
    
        for name_elems, dist_elems, rate_elems, tr_elems, or_elems, pr_elems, dr_elems, fac_elems, de1_elems, de2_elems, de3_elems in zip(name_elems, dist_elems, rate_elems, tr_elems, or_elems, pr_elems, dr_elems, fac_elems, de1_elems, de2_elems, de3_elems):
            try:
                names.append(name_elems.text)
                distance.append(dist_elems.text.replace("Hostel - ", ""))
                ratings.append(rate_elems.text)
                t_rev.append(tr_elems.text.replace("Total Reviews", ""))
                o_rev.append(or_elems.text)
                privates.append(pr_elems.text.replace("Rs", "₹."))
                dorms.append(dr_elems.text.replace("Rs", "₹."))
                faci.append(fac_elems.text.replace("\n", ", "))
                fin = (f"{de1_elems.text}, {de2_elems.text}, {de3_elems.text}")
                desc.append(fin)
            except NoSuchElementException:
                names.append("-")
                distance.append("-")
                ratings.append("-")
                t_rev.append("-")
                o_rev.append("-")
                privates.append("-")
                dorms.append("-")
                faci.append("-")
                desc.append("-")
                
        time.sleep(5)
        d.close()

    df = pd.DataFrame({'Name':names, 'Distance':distance, 'Ratings':ratings, 'Total Reviews':t_rev, 
                       'Overall Reviews':o_rev, 'Privates':privates, 'Dorms': dorms, 'Facilities':faci, 'Description' : desc
                        })
    #     df.to_csv(file_path, sep=',')

    return d, df

In [37]:
d, data = get_hostels_test("https://www.hostelworld.com/")
d.quit()

In [38]:
data.head(15)

,Name,Distance,Ratings,Total Reviews,Overall Reviews,Privates,Dorms,Facilities,Description
0,London Waterloo Hostel,0.7km from city centre,7.2,2419,Very Good,₹.6463,₹.1547,"Free WiFi, Free Breakfast, Follows Covid-19 sa...","Perfect Location, Superb Staff, Awesome Cleanl..."
1,St Christopher's Inn - London Bridge,1.8km from city centre,7.0,3237,Very Good,₹.4450,₹.1254,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Marvellous Staff, Excellent ..."
2,Barmy Badger Backpackers,5.5km from city centre,9.7,1659,Superb,₹.6068,₹.2124,"Free WiFi, Free Breakfast","Perfect Location, Superb Staff, Fantastic Clea..."
3,Wombat's The City Hostel London,3.6km from city centre,8.8,13191,Fabulous,₹.8075,₹.1852,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
4,Prime Backpackers Angel,3.6km from city centre,10,514,Superb,₹.8790,₹.2393,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
5,St Christopher's Village,1.8km from city centre,8.7,10854,Fabulous,₹.6877,₹.1305,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Fantastic Clea..."
6,"PubLove @ The Steam Engine, Waterloo",0.5km from city centre,8.5,194,Fabulous,₹.19418,₹.1618,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Superb Staff, Excellent Clea..."
7,Smart Camden Inn Hostel,4.4km from city centre,7.6,2687,Very Good,₹.15474,₹.1293,"Free WiFi, Follows Covid-19 sanitation guidance","Wonderful Location, Superb Staff, Excellent Cl..."
8,"PubLove @ The Crown, Battersea",4.7km from city centre,6.8,214,Good,₹.6372,₹.1618,"Free WiFi, Follows Covid-19 sanitation guidance","Wonderful Location, Marvellous Staff, Excellen..."
9,Safestay London Elephant & Castle,1.7km from city centre,8.0,3958,Fabulous,₹.10316,₹.1820,"Free WiFi, Follows Covid-19 sanitation guidance","Perfect Location, Marvellous Staff, Fantastic ..."
